# Prompt Engineering, explored with Semantic Kernel and Azure OpenAI

To quickly get started, follow these steps:

1. Install [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) in VSCode.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the parent folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.


> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Background

## What is Prompt Engineering?
Prompt engineering is an iterative approach for crafting and refining prompts to enhance interactions with Large Language Models (LLMs). Mastery of prompt engineering is key to unlocking the full potential of LLMs in various applications. This has been pivotal in achieving advanced use cases in Microsoft Copilots.

This notebook serves as your go-to resource for effective prompt engineering techniques.

### Best Practices: Insights from Azure OpenAI

#### Be Specific and Descriptive
Craft your prompts to be both specific and descriptive to minimize ambiguity. Using analogies or metaphors can aid in making the prompts more understandable and relatable to the model.

#### Be Repetitive
- **Repeat**: Reiterate key instructions to ensure clarity and focus in the model's output.
- **Order Matters**: The sequence in which you present instructions can influence the model’s response due to its recency bias.

#### Space Efficiency
- **Token Limitations**: Be aware of the token limits for the model you are invoking.
- **Data Formats**: Opt for tabular formats over JSON for greater space efficiency.
- **White Space**: Use space judiciously, as each extra space counts as a token and can limit the model's performance.



## Let's Get Started

We will be utilizing **Semantic Kernel** to orchestrate interactions with the `gpt-35-turbo` model, which is deployed on **Azure OpenAI** for brevity. Alternatively, you can also use the **Azure OpenAI SDK** for model orchestration.


## Load settings from .env file

In [ ]:
#r "nuget: dotenv.net, 3.1.2"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

## Prepare kernel using Azure Cognitive Search

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230718.1-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureSearch, 0.17.230718.1-preview"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Memory.AzureSearch;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Text;
using System.IO;
using System.Text.Json;

var kernel = Kernel.Builder
    
    // Use Azure Cognitive Search for the kernel Memory
    .WithMemoryStorage(new AzureSearchMemoryStore(
        env["AZURE_SEARCH_ENDPOINT"],
        env["AZURE_SEARCH_API_KEY"]))

    // Use Azure OpenAI for Embeddings (model: text-embedding-ada-002)
    .WithAzureTextEmbeddingGenerationService(
        deploymentName: "text-embedding-ada-002",
        endpoint: env["AZURE_OPENAI_ENDPOINT"],
        apiKey: env["AZURE_OPENAI_API_KEY"])

    // Use Azure OpenAI for Semantic Functions (model = gpt-35-turbo)
    .WithAzureChatCompletionService(
        deploymentName: "gpt-35-turbo",
        endpoint: env["AZURE_OPENAI_ENDPOINT"],
        apiKey: env["AZURE_OPENAI_API_KEY"])
        
    .Build();

## Vectorize and persist embeddings in Azure Cognitive Search with Semantic Kernel

In [ ]:
var dataset = "intelligent-investor.txt";
var recommendationServicePath = "../../../../services/recommendation-service/dotnet";
const int MaxTokensPerParagraph = 160;
const int MaxTokensPerLine = 60;

// Read file from local file system
var filePath = Path.Combine(recommendationServicePath, "resources", "sample-datasets", dataset);
var streamReader = new StreamReader(filePath);
var text = await streamReader.ReadToEndAsync();

// Chunk, generate embeddings, and persist to vectordb
var memoryCollectionName = "userId";

var lines = TextChunker.SplitPlainTextLines(text, MaxTokensPerLine);
var chunks = TextChunker.SplitPlainTextParagraphs(lines, MaxTokensPerParagraph);

for (var i = 0; i < chunks.Count; i++)
{
    var chunk = chunks[i];
    var key = await kernel.Memory.SaveInformationAsync(
        memoryCollectionName,
        chunk,
        $"{dataset}-{i}",
        $"Dataset: {dataset} Chunk: {i}",
        i.ToString());
}
System.Console.WriteLine($"Saved {chunks.Count} chunks to memory collection {memoryCollectionName}");

## Search and retrieve documents using Semantic Kernel

In [ ]:
var query = "Ben Graham's investment philosophy";
Console.WriteLine(query + "\n");

var results = kernel.Memory.SearchAsync(collection: memoryCollectionName, query, limit: 2);
await foreach(var result in results)
{
    Console.WriteLine("   " + result.Metadata.Text);
    Console.WriteLine("   Relevance: " + result.Relevance + "\n");
}

## Grounding Miyagi prompts with SK's Memory "recall"

> Note that this prompt template (semantic function) is located under services/reccommendation-service/dotnet/plugins/AdvisorPlugin

In [ ]:
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.SkillDefinition;

// recall is from the TextMemorySkill, which does the retrieval step
kernel.ImportSkill(new TextMemorySkill());

var pluginFolder = $"{recommendationServicePath}/plugins";
var advisorPlugin = kernel.ImportSemanticSkillFromDirectory(pluginFolder, "AdvisorPlugin");
advisorPlugin

### Set context variables

In [ ]:
var context = kernel.CreateNewContext();

// Set the parameters for the TextMemorySkill
context[TextMemorySkill.CollectionParam] = memoryCollectionName;
context[TextMemorySkill.RelevanceParam] = "0.8";
context[TextMemorySkill.LimitParam] = "3";

// Set the parameters for the AdvisorPlugin
var stocks = new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
};
context["stocks"] = JsonSerializer.Serialize(stocks);

context["userId"] = "50";
context["voice"] = "Jim Cramer";
context["risk"] = "aggressive";

context

### Create native function

In [ ]:
using System.ComponentModel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.SkillDefinition;

/// <summary>
///     UserProfilePlugin shows a native skill example to look user info given userId.
/// </summary>
/// <example>
///     Usage: kernel.ImportSkill("UserProfilePlugin", new UserProfilePlugin());
///     Examples:
///     SKContext["userId"] = "000"
///     {{UserProfilePlugin.GetUserAge $userId }} => {userProfile}
/// </example>
public class UserProfilePlugin
{
    /// <summary>
    ///     Name of the context variable used for UserId.
    /// </summary>
    public const string UserId = "UserId";

    private const string DefaultUserId = "40";
    private const int DefaultAnnualHouseholdIncome = 150000;
    private const int Normalize = 81;

    /// <summary>
    ///     Lookup User's age for a given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetUserAge")]
    [Description("Given a userId, get user age")]
    public string GetUserAge(
        [Description("Unique identifier of a user")]
        string userId,
        SKContext context)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        int age;

        if (int.TryParse(userId, out var parsedUserId))
            age = parsedUserId > 100 ? parsedUserId % Normalize : parsedUserId;
        else
            age = int.Parse(DefaultUserId);

        // invoke a service to get the age of the user, given the userId
        return age.ToString();
    }

    /// <summary>
    ///     Lookup User's annual income given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    /// <param name="context">Contains the context variables.</param>
    [SKFunction]
    [SKName("GetAnnualHouseholdIncome")]
    [Description("Given a userId, get user annual household income")]
    public string GetAnnualHouseholdIncome(
        [Description("Unique identifier of a user")]
        string userId,
        SKContext context)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        var random = new Random();
        var randomMultiplier = random.Next(1000, 8000);

        // invoke a service to get the annual household income of the user, given the userId
        var annualHouseholdIncome = int.TryParse(userId, out var parsedUserId)
            ? parsedUserId * randomMultiplier
            : DefaultAnnualHouseholdIncome;

        return annualHouseholdIncome.ToString();
    }
}

In [ ]:
// import the UserProfilePlugin
kernel.ImportSkill(new UserProfilePlugin(), "UserProfilePlugin");

### Invoke the LLM

In [ ]:
var result = await kernel.Func("AdvisorPlugin", "InvestmentAdvise").InvokeAsync(context);

Console.WriteLine(result);


![RaG Workflow](../../../../assets/images/sk-memory-orchestration.png)